<img src="../../img/python-logo-no-text.svg"
     style="display:block;margin:auto;width:10%"/>
<br>
<div style="text-align:center; font-size:200%;">
  <b>Generator-Funktionen und Coroutinen</b>
</div>
<br/>
<div style="text-align:center;">Dr. Matthias Hölzl</div>
<br/>
<div style="text-align:center;">module_130_functions/topic_370_e4_generator_functions</div>

## Generator Funktionen

Komplexere Fälle können von Generator Expressions nicht mehr abgedeckt werden.

- Generator, der alle Zahlen erzeugt (ohne Obergrenze)
- Generator, der ein Iterable modifiziert (z.B. mehrfach ausführt, eine fixe Anzahl
  an Elementen nimmt)

Für diese Fälle gibt es Generator-Funktionen


Mit dem Schlüsselwort `yield` kann eine Funktion "mehrere Werte zurückgeben".
Eine Funktion, die `yield` in ihrem Rumpf verwendet, wird Generatorfunktion
genannt. Ein Aufruf einer Generatorfunktion wertet nicht den Rumpf der
Funktion aus, sondern es wird ein *Generator* zurückgegeben, der mehrmals
einen Wert zurückgeben kann:

In [ ]:
def integers(start=0):
    n = start
    while True:
        yield n
        n += 1

In [ ]:
gen = integers()
print(repr(gen))
print(repr(iter(gen)))

In [ ]:
gen = integers()

In [ ]:
next(gen)

In [ ]:
next(gen)

In [ ]:
for i in integers():
    if i > 3:
        break
    print(i, end=" ")


Schreiben Sie eine Generatorfunktion `one_based_range(n)`, die `range(n)` für
ein Argument simuliert, aber von 1 bis einschließlich `n` iteriert.

In [ ]:
def one_based_range(n):
    for i in range(1, n + 1):
        yield i

In [ ]:
assert list(one_based_range(4)) == [1, 2, 3, 4]


Schreiben Sie eine Generatorfunktion `inklusive_range()`, die die komplette
Funktionalität von `range()` simuliert (d.h. die mit einem, zwei oder drei
Argumenten aufgerufen werden kann), aber ihre obere Grenze einschließt und die
Iteration von 1 beginnt, wenn keine untere Grenze angegeben wird.

Stellen Sie sicher, dass Ihre Implementierung die vorgegebenen Testfälle
erfüllt.

In [ ]:
def inclusive_range(m, n=None, step=1):
    if n is None:
        assert step == 1, "Cannot specify step when no upper bound is specified."
        n = m
        m = 1
    for i in range(m, n + 1, step):
        yield i

In [ ]:
assert list(inclusive_range(3)) == [1, 2, 3]

In [ ]:
assert list(inclusive_range(2, 4)) == [2, 3, 4]

In [ ]:
assert list(inclusive_range(2, 2)) == [2]

In [ ]:
assert list(inclusive_range(2, 1)) == []

In [ ]:
assert list(inclusive_range(2, 6, 2)) == [2, 4, 6]


Wir können Generatorfunktionen verwenden, um Funktionen zu schreiben, die
Iteratoren verarbeiten. Beispielsweise nimmt die Generatorfunktion `take()`
eine feste Anzahl von Werten von einem Iterator:

In [ ]:
def take(n, it):
    for i in range(n):
        yield next(it)

In [ ]:
list(take(3, integers()))


Beachten Sie, dass die Funktion `drop()`, die die ersten `n` Elemente eines
Iterators entfernt, keine Generatorfunktion, sondern eine reguläre Funktion
ist:

In [ ]:
def drop(n, it):
    for i in range(n):
        next(it)
    return it

In [ ]:
list(take(3, drop(2, integers())))


Mit Generatorfunktionen können wir auch komplexere Iterations-Operationen
definieren:

In [ ]:
def repeat_n_times(n, it):
    for _ in range(n):
        for elt in it:
            yield elt

In [ ]:
for num in repeat_n_times(3, range(5)):
    print(num, end=" ")

In [ ]:
def repeat_n_times_2(n, it):
    for _ in range(n):
        yield from it

In [ ]:
for num in repeat_n_times_2(3, range(5)):
    print(num, end=" ")


## Mini-Workshop "Generator"

Schreiben Sie einen Generator `leaves(tree: Bintree)`, der alle Blätter eines
Binärbaums zurückgibt (d.h., alle Knoten, die nicht Instanzen von `Bintree` sind).

In [ ]:

from dataclasses import dataclass
from typing import Optional

In [ ]:
@dataclass
class Bintree:
    value: object = None
    left: Optional["Bintree"] = None
    right: Optional["Bintree"] = None

In [ ]:
def leaves(tree: Bintree):
    if tree:
        yield from leaves(tree.left)
        yield tree.value
        yield from leaves(tree.right)

In [ ]:
tree1 = Bintree(1)
tree2 = Bintree(2, Bintree(1), Bintree(3))
tree3 = Bintree(1, Bintree(0),
                Bintree(3, Bintree(2), Bintree(5, Bintree(4), Bintree(6))))

In [ ]:
assert tuple(leaves(tree1)) == (1,)

In [ ]:
assert tuple(leaves(tree2)) == (1, 2, 3)

In [ ]:
assert tuple(leaves(tree3)) == tuple(range(7))


# Coroutinen

`yield` kann auch verwendet werden, um einen Wert an die Stelle zurückzugeben,
an der es "aufgerufen" wird. In diesem Fall nennen wir den Generator auch eine
*Coroutine*.

Coroutinen sind nützliche Bausteine für Features wie z.B. kooperatives
Multitasking oder Event-basierte Programmierung.

Um eine Coroutine `c` zu starten, rufen wir die Methode `c.send(None)` auf.
Zum Senden nachfolgender Werte verwenden wir `c.send(value)`.


In [ ]:
def my_coroutine(n):
    for i in range(n):
        x = yield
        print(x)

In [ ]:
c = my_coroutine(3)
print(c)

In [ ]:
# Initial value must be `None`
c.send(None)

In [ ]:
c.send(10)

In [ ]:
c.send(20)

In [ ]:
# c.send(40)

In [ ]:
def your_coroutine(n):
    for i in range(1, n + 1):
        x = (yield i)
        print("your_coroutine:", x)

In [ ]:
c = your_coroutine(3)

In [ ]:
_x = c.send(None)
print("top level:", _x)

In [ ]:
try:
    while True:
        _x = c.send(_x * 10)
        print("top level:", _x)
except StopIteration:
    print("Done.")

In [ ]:
def counter(max_value):
    i = 0
    while i < max_value:
        val = yield i
        if val is not None:
            i = val
        else:
            i += 1

In [ ]:
it = counter(100)

In [ ]:
next(it)

In [ ]:
next(it)

In [ ]:
it.send(98)

In [ ]:
next(it)

In [ ]:
# next(it)